In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import keras

Using TensorFlow backend.


In [5]:
from keras import backend as K

In [6]:
from keras.models import Sequential

In [7]:
from keras.layers import Dense

In [8]:
from sklearn import model_selection

In [9]:
import math

In [10]:
import tensorflow as tf

In [11]:
import horovod.keras as hvd

In [12]:
from liz_models import distance_generation

In [13]:
from liz_models import weights_generation

In [14]:
df = pd.DataFrame()

In [15]:
df['distances'] = distance_generation()

In [16]:
df['weights'] = weights_generation()

In [17]:
df[['distances','weights']].count()

distances    100000
weights      100000
dtype: int64

In [18]:
df['prices'] = np.array([((df['distances'][i] * df['weights'][i])/100) for i in range(len(df['distances']))])

In [19]:
df.head()

,distances,weights,prices
0,120,483,579.60
1,466,457,2129.62
2,746,92,686.32
3,846,209,1768.14
4,810,218,1765.80


In [20]:
hvd.init()

0

In [21]:
config = tf.ConfigProto()

In [22]:
config.gpu_options.allow_growth = True

In [23]:
config.gpu_options.visible_device_list = str(hvd.local_rank())

In [24]:
K.set_session(tf.Session(config=config))

In [25]:
batch_size = 128

In [26]:
num_classes = 10

In [27]:
epochs = int(math.ceil(120.0) / hvd.size())

In [28]:
x, x_val, y, y_val = model_selection.train_test_split(df['prices'], df['distances'], test_size=0.2, random_state=42)

In [29]:
dlce = lambda l: [v for v in l if v >= 0]

In [30]:
x = np.array(dlce(x))
x_val = np.array(dlce(x_val))

In [31]:
y = np.array(dlce(y))
y_val = np.array(dlce(y_val))

In [32]:
opt = keras.optimizers.RMSprop(1.0 * hvd.size())

In [33]:
opt = hvd.DistributedOptimizer(opt)

In [34]:
model = Sequential()

In [35]:
model.add(Dense(1, input_dim=1))

In [36]:
model.compile(optimizer=opt, loss=keras.losses.mse,metrics=['accuracy'])

In [37]:
callbacks = [hvd.callbacks.BroadcastGlobalVariablesCallback(0)]

In [38]:
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoints/checkpoint-{epoch}.h5'))

In [39]:
model.fit(x, y, batch_size=batch_size, callbacks=callbacks, epochs=epochs, verbose=1, validation_data=(x_val, y_val))

Train on 80000 samples, validate on 20000 samples
Epoch 1/120
80000/80000 [==============================] - 1s 12us/step - loss: 830986.2601 - acc: 0.0010 - val_loss: 59484.1354 - val_acc: 0.0023
Epoch 2/120
80000/80000 [==============================] - 1s 13us/step - loss: 804842.2723 - acc: 6.7500e-04 - val_loss: 676696.5997 - val_acc: 8.0000e-04
Epoch 3/120
80000/80000 [==============================] - 1s 13us/step - loss: 809121.1054 - acc: 7.6250e-04 - val_loss: 133991.1151 - val_acc: 9.5000e-04
Epoch 4/120
80000/80000 [==============================] - 1s 12us/step - loss: 811264.6537 - acc: 6.6250e-04 - val_loss: 901047.3252 - val_acc: 6.5000e-04
Epoch 5/120
80000/80000 [==============================] - 1s 13us/step - loss: 792805.2952 - acc: 7.2500e-04 - val_loss: 105007.1941 - val_acc: 1.0000e-03
Epoch 6/120
80000/80000 [==============================] - 1s 13us/step - loss: 797787.6206 - acc: 6.5000e-04 - val_loss: 417244.0006 - val_acc: 7.0000e-04
Epoch 7/120
80000/80000

80000/80000 [==============================] - 1s 13us/step - loss: 797864.4970 - acc: 4.3750e-04 - val_loss: 222083.9217 - val_acc: 1.0000e-03
Epoch 54/120
80000/80000 [==============================] - 1s 12us/step - loss: 803609.6212 - acc: 4.6250e-04 - val_loss: 895972.3816 - val_acc: 6.5000e-04
Epoch 55/120
80000/80000 [==============================] - 1s 13us/step - loss: 801283.6728 - acc: 6.6250e-04 - val_loss: 1570675.3116 - val_acc: 3.5000e-04
Epoch 56/120
80000/80000 [==============================] - 1s 11us/step - loss: 801626.9940 - acc: 4.1250e-04 - val_loss: 116526.6359 - val_acc: 0.0011
Epoch 57/120
80000/80000 [==============================] - 1s 11us/step - loss: 799582.7240 - acc: 5.8750e-04 - val_loss: 448154.7834 - val_acc: 1.0000e-03
Epoch 58/120
80000/80000 [==============================] - 1s 11us/step - loss: 809440.9347 - acc: 6.6250e-04 - val_loss: 99556.2404 - val_acc: 0.0014
Epoch 59/120
80000/80000 [==============================] - 1s 15us/step - loss

80000/80000 [==============================] - 1s 11us/step - loss: 796776.4210 - acc: 8.1250e-04 - val_loss: 2604540.6016 - val_acc: 5.0000e-05
Epoch 106/120
80000/80000 [==============================] - 1s 11us/step - loss: 808862.4151 - acc: 5.3750e-04 - val_loss: 42414.2667 - val_acc: 0.0019
Epoch 107/120
80000/80000 [==============================] - 1s 11us/step - loss: 795105.5116 - acc: 5.7500e-04 - val_loss: 350558.1299 - val_acc: 6.5000e-04
Epoch 108/120
80000/80000 [==============================] - 1s 11us/step - loss: 798398.0369 - acc: 5.8750e-04 - val_loss: 709862.8768 - val_acc: 3.5000e-04
Epoch 109/120
80000/80000 [==============================] - 1s 11us/step - loss: 796605.6882 - acc: 9.1250e-04 - val_loss: 81466.3468 - val_acc: 0.0014
Epoch 110/120
80000/80000 [==============================] - 1s 10us/step - loss: 795932.3791 - acc: 5.3750e-04 - val_loss: 1452340.9096 - val_acc: 2.5000e-04
Epoch 111/120
80000/80000 [==============================] - 1s 11us/step 

In [40]:
score = model.evaluate(x_val, y_val, verbose=0)

In [41]:
print('Test loss:', score[0])

Test loss: 383646.78595


In [42]:
print('Test accuracy:', score[1])

Test accuracy: 0.0008


In [43]:
print(model.predict(np.array([np.random.randint(10,100) for _ in range(5)])))

[[211.01057]
 [220.68555]
 [214.1012 ]
 [214.1012 ]
 [211.9512 ]]


In [44]:
model.save_weights('travel-price.hdf5')

In [45]:
with open('travel-price.json','w') as f:
    f.write(model.to_json())
    f.close()